In [ ]:
# modelagem de dados

import pandas as pd
import numpy as np

# libs gráfica
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt

# avisos
import warnings
warnings.filterwarnings('ignore')

# configurando pandas
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 100)

# configuração matplot
plt.rcParams['figure.figsize'] = (15,6)
plt.style.use('ggplot')

# **Importação dos dados**


In [ ]:
# Dados
basedados = pd.read_csv('/clean_data.csv')

basedados.shape

In [ ]:
# verificar dados
basedados.head()

In [ ]:
# info
basedados.info()

In [ ]:
# nulos
basedados.isnull().sum()

In [ ]:
# campos unicos
basedados.nunique()

In [ ]:
# remover coluna
basedados.drop(columns = [ 'Unnamed: 0'], inplace=True)

In [ ]:
basedados.head()

## **Análise macros**

## **Prefeitos eleitos no País**

In [ ]:
# campos nulos
print(basedados['job'].unique())
print(basedados['elector_count'].unique())

In [ ]:
# Aplicando filtro

query_prefeitos = basedados[
    (basedados['job'] == 'prefeito') &
    (basedados['elector_count'] == 's')
]

# dimensão
query_prefeitos.shape

In [ ]:
# Agrupando quantidade de votos por partido
votosPrefeitoPartido = query_prefeitos.groupby(by = ['main_party']).agg(
    quantidade = ('candidate_vote_count', 'count')
)

votosPrefeitoPartido.head()

In [ ]:
# quantidade de prefeitos eleitos
qtdPrefeitosEleitos = votosPrefeitoPartido['quantidade'].sum()

qtdPrefeitosEleitos

In [ ]:
# gerando coluna %
votosPrefeitoPartido['%'] = votosPrefeitoPartido['quantidade'] / qtdPrefeitosEleitos * 100
votosPrefeitoPartido['%'] = round(votosPrefeitoPartido['%'],2)

# ordenar
votosPrefeitoPartido.sort_values('quantidade', inplace = True, ascending = False)

votosPrefeitoPartido.head()

In [ ]:
# paleta de cores
sns.color_palette('magma',len(votosPrefeitoPartido))

In [ ]:
# tamanho
plt.figure( figsize = (20,6))

# Paleta
paletaCores = sns.color_palette('magma',len(votosPrefeitoPartido))

# plot
plt.bar(
    votosPrefeitoPartido.index,
    votosPrefeitoPartido['quantidade'],
    width=0.9,
    color=paletaCores 
    );

# Título
plt.title('Prefeitos eleitos no país', loc='left', fontsize=20, color='#404040', fontweight = 600);

# Labels
plt.ylabel('Quantidade de prefeitos')
plt.xlabel('Partidos')
plt.xticks(rotation=90);

# Ajuste de escala do gráfico
plt.ylim(0,
         votosPrefeitoPartido['quantidade'].max() + (votosPrefeitoPartido['quantidade'].max() * 0.1)
         )

# incluindo os dados no gráfico
for posicao, valor in enumerate(votosPrefeitoPartido['quantidade']):
  plt.text(
      # posicao x, y
      posicao -0.3,
      valor +10,
      # valor
      valor,
      # paleta
      color = paletaCores[posicao], 
      # tamanho
      size=10,
      # espesura
      fontweight = 700
  )

  totalEleitos =votosPrefeitoPartido['quantidade'].sum()

  # info complementar
  plt.annotate(
      f'total eleitos no Brasil: {totalEleitos}',
      xy=(0.99,0.94),
      xycoords='axes fraction',
      ha='right',
      va='center',
      color='green',
      fontsize=14,
      fontweight=100,
      bbox=dict(facecolor='#ffffff', edgecolor='green',boxstyle='round', pad=0.25)
  )


## **Veriadores eleitos no País**

In [ ]:
basedados

In [ ]:
# Aplicando filtro

query_vereadores = basedados[
    (basedados['job'] == 'vereador') &
    (basedados['elector_count'] == 's')
]

# dimensão
query_vereadores.shape

In [ ]:
# Agrupando quantidade de votos por partido
votosVereadoresPartido = query_vereadores.groupby(by = ['main_party']).agg(
    quantidade = ('candidate_vote_count', 'count')
)

votosVereadoresPartido.head()

In [ ]:
# quantidade de Veriadores eleitos
qtdVereadoresEleitos = votosVereadoresPartido['quantidade'].sum()

qtdVereadoresEleitos

In [ ]:
# gerando coluna %
votosVereadoresPartido['%'] = votosVereadoresPartido['quantidade'] / qtdVereadoresEleitos * 100
votosVereadoresPartido['%'] = round(votosVereadoresPartido['%'],2)

# ordenar
votosVereadoresPartido.sort_values('quantidade', inplace = True, ascending = False)

votosVereadoresPartido.head()

In [ ]:
# Plot
plt.hlines(
    y=votosVereadoresPartido.index,
    xmin=0,
    xmax=votosVereadoresPartido['quantidade'],
    # Espesura da linha
    lw = 5,
    # Cores
    color=paletaCores,
    alpha=0.5
);

# plot de indicadores
plt.scatter(
    # Dados
    votosVereadoresPartido['quantidade'],
    votosVereadoresPartido.index,
    # Tamanho
    s=100,
    # cor
    color=paletaCores,
    # Transparencia
    alpha=0.8
)

totalEleitos =votosVereadoresPartido['quantidade'].sum()

  # info complementar
plt.annotate(
    f'total eleitos no Brasil: {totalEleitos}',
    xy=(0.99,0.94),
    xycoords='axes fraction',
    ha='right',
    va='center',
    color='green',
    fontsize=14,
    fontweight=100,
    bbox=dict(facecolor='#ffffff', edgecolor='green',boxstyle='round', pad=0.25)
  )

plt.title('Vereadores eleitos no País', loc='left', fontsize=20, color='#404040', fontweight=500 );

## **Análise de Correlação**

In [ ]:
# Verificar dimensão igual
len(votosPrefeitoPartido), len(votosVereadoresPartido)

In [ ]:
# Tabela
tabCorrelacao = votosPrefeitoPartido['quantidade'].reset_index()

# Cruzamento
tabCorrelacao = pd.merge(tabCorrelacao, votosVereadoresPartido.reset_index(), on=['main_party'], how='inner')

# Ajustar
tabCorrelacao.columns = ['Partido', 'Prefeitos', 'Vereadores', '%']

# Apagar
tabCorrelacao.drop(columns=['%'], inplace=True)


tabCorrelacao

In [ ]:
sns.regplot(
    # dados
    x = tabCorrelacao['Prefeitos'],
    y = tabCorrelacao['Vereadores'],
    # intervalo de confiança para estimar a regressão
    ci = 95,
    # cores
    scatter_kws={
        'color': 'blue',
    's':80,
    'alpha':0.5,
    },
    # cor da linha
    line_kws={
        'color' :'green',
        'alpha': 0.2,
        'lw':2
    },
);

# título
plt.title('Prefeitos x Vereadores');

for linha in range(0,tabCorrelacao.shape[0]):
  plt.text(
      tabCorrelacao['Prefeitos'][linha] + 0.8,
      tabCorrelacao['Vereadores'][linha],
      tabCorrelacao['Partido'][linha],
      size='medium',
      color='gray',
      weight='semibold'
  )

## **Plot tridimensional**

In [ ]:
quantidadeCandidatos = basedados.groupby(by=['main_party']).count().iloc[:,1].reset_index()

quantidadeCandidatos.columns = ['Partido','Candidatos']

quantidadeCandidatos.sort_values('Candidatos', inplace = True, ascending = False)

quantidadeCandidatos.head()

In [ ]:
# Cruzar
tabCorrelacao = pd.merge(tabCorrelacao, quantidadeCandidatos, on=['Partido'], how='inner')

# Verificar
tabCorrelacao.head()

In [ ]:
# Correlação
tabCorrelacao.corr()

In [ ]:
# Plot Tridimensional

# Tamanho
Figura = plt.figure( figsize=(15, 8) )

# Instanciar
Eixo = Figura.add_subplot( 111, projection='3d' )

# Plot
Eixo.scatter(
    tabCorrelacao['Prefeitos'],
    tabCorrelacao['Vereadores'],
    tabCorrelacao['Candidatos'],
    c='black',
    s=100
)

# Rotacao
Eixo.view_init(30, 185)

# Labels
Eixo.set_xlabel('Prefeitos')
Eixo.set_ylabel('Vereadores')
Eixo.set_zlabel('Cadidatos');

In [ ]:
Figura = px.scatter_3d(
    tabCorrelacao,
    x='Prefeitos',
    y='Vereadores',
    z='Candidatos',
    color='Partido',
    opacity=0.7
)

Figura.show()